<h2>Calculates time series of partitioned areas by major basin</h2>


In [1]:
%pylab notebook
import csv
import datetime as dt
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as md
from nose.tools import set_trace
from charistools.convertors import Dem2Hypsometry
from charistools.hypsometry import Hypsometry
from charistools.meltModels import TriSurfTempIndexMelt
from charistools.meltModels import ImshowTriSurfMelt
from charistools.meltModels import PlotTriSurfInput
from charistools.meltModels import PlotTriSurfMelt
from charistools.modelEnv import ModelEnv
from charistools.timeSeries import TimeSeries
import pandas as pd
import re
import os

/Users/brodzik/.conda/envs/charis/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [14]:
snowyConfigFile = '/Users/brodzik/ipython_notebooks/charis/snowy_basins_modelEnv_config.ini'
snowyBasinEnv = ModelEnv(tileConfigFile=snowyConfigFile)

In [4]:
#snowyBasinEnv.tileConfig

In [5]:
# ids = ['AM', 'BR', 'GA_v01', 'IN_v01', 'SY_v01']
ids = ['AM']

In [6]:
# Generate a list of all OBJECTID basins
def objectIDs(id):
    %cd /Users/brodzik/projects/CHARIS/derived_hypsometries/MODSCAG_GF_v09_fromFile_rainv01_less_ET/
    objectIDs = []
    ids = glob.glob("%s_OBJECTID*" % id)
    ids.sort()
    print("There are %d OBJECTID sub-basins for the %s major basin" % (len(ids), id))
    return(ids)


In [22]:
years = np.arange(1) + 2001
years

array([2001])

In [27]:
for year in years:
    for i, id in enumerate(ids):
        
        # Get a list of the snowy_basins for this id
        # Read each area file for each objectID in turn, and sub-total by doy
        objects = objectIDs(id)
        #objects = objects[:2]
        print("subbasins for this major basin:")
        print(objects)
        partition_types = ['snow_on_land', 'exposed_glacier_ice', 'snow_on_ice']
        sol = pd.DataFrame()
        soi = pd.DataFrame()
        egi = pd.DataFrame()
        for object in objects:
            print("Next objectID: %s" % object)
            SOLFile = snowyBasinEnv.hypsometry_filename(
                type='snow_on_land_melt_by_elevation',
                drainageID=object,
                year=year,
                modice_nstrikes=3)
            SOIFile = snowyBasinEnv.hypsometry_filename(
                type='snow_on_ice_melt_by_elevation',
                drainageID=object,
                year=year,
                modice_nstrikes=3,
                threshold='fromFile',
                ablation_method='grsize_scag')
            EGIFile = snowyBasinEnv.hypsometry_filename(
                type='exposed_glacier_ice_melt_by_elevation',
                drainageID=object,
                year=year,
                modice_nstrikes=3,
                threshold='fromFile',
                ablation_method='grsize_scag')
            
            solHyps = Hypsometry(SOLFile)
            soiHyps = Hypsometry(SOIFile)
            egiHyps = Hypsometry(EGIFile)
            
            # If SOLmelt is non-empty, but SOImelt and SGImelt are empty,
            # then make a copy of the dimensions of SOLmelt that is filled with zeroes
            if not solHyps.data.empty and soiHyps.data.empty and egiHyps.data.empty:
                soiHyps.data = solHyps.data.copy()
                soiHyps.data[:] = 0.
                egiHyps.data = solHyps.data.copy()
                egiHyps.data[:] = 0.
            
            sol_by_doy = solHyps.data_by_doy()
            sol_by_doy.name = object
            soi_by_doy = soiHyps.data_by_doy()
            soi_by_doy.name = object
            egi_by_doy = egiHyps.data_by_doy()
            egi_by_doy.name = object

            sol = sol.append(sol_by_doy)
            soi = soi.append(soi_by_doy)
            egi = egi.append(egi_by_doy)
            
        print(sol)
        print(soi)
        print(egi)
        
        # Make a new DataFrame with the volumes by date and SOL/SOI/EGI columns
        df = pd.concat([sol.sum(), egi.sum(), soi.sum()], axis=1)
        df.columns = ['SOL_melt_km3', 'EGI_melt_km3', 'SOI_melt_km3']

        print(df)
        # df['Snow_ice_area_km2'] = df.sum(axis=1)
        # df['Full_basin_area_km2'] = full_basin_area
        # df['Full_basin_less_snow_ice_area_km2'] = full_basin_area - df['Snow_ice_area_km2']
        # df['Snowy_basin_area_km2'] = snowy_basin_area
        # df['Snowy_basin_less_snow_ice_area_km2'] = snowy_basin_area - df['Snow_ice_area_km2']

        # Write out the data frame
        #outFile = os.path.join(
        #    "/Users/brodzik/projects/CHARIS/derived_hypsometries/MODSCAG_GF_v09_fromFile_rainfall_less_ET",
        #    "%s.%4d.partition_area.new.txt" % (id, year))
        #print("Outfile will be: %s" % outFile)
        
        #fh = open(outFile, 'w')
        #fh.write("# Major_basin: %s\n" % id)
        #fh.write("# Snowy basins: %s\n" % (", ".join(objects)))
        #fh.write("# Number_of_snowy_basins: %d\n" % (len(objects)))
        #fh.write(str(len(df.columns)) + '\n')
        #fh.write(' '.join([str(col) for col in df.columns]) + '\n')
        #fh.close()
        
        #df['yyyy'] = df.index.year
        #df['mm'] = df.index.month
        #df['dd'] = df.index.day
        #df['doy'] = df.index.dayofyear
        #df = df.reindex_axis(
        #    ['yyyy', 'mm', 'dd', 'doy'] +
        #    list(df.columns[:-4]), axis=1)
        
        #format = "%.4f"                           
        #df.to_csv(outFile, mode='a', header=False, index=False, sep=" ",  
        #          float_format=format, quoting=csv.QUOTE_NONE) 
            
            

/Users/brodzik/projects/CHARIS/derived_hypsometries/MODSCAG_GF_v09_fromFile_rainv01_less_ET
There are 27 OBJECTID sub-basins for the AM major basin
subbasins for this major basin:
['AM_OBJECTID100', 'AM_OBJECTID62', 'AM_OBJECTID64', 'AM_OBJECTID65', 'AM_OBJECTID66', 'AM_OBJECTID68', 'AM_OBJECTID69', 'AM_OBJECTID72', 'AM_OBJECTID77', 'AM_OBJECTID78', 'AM_OBJECTID79', 'AM_OBJECTID80', 'AM_OBJECTID81', 'AM_OBJECTID82', 'AM_OBJECTID83', 'AM_OBJECTID84', 'AM_OBJECTID85', 'AM_OBJECTID86', 'AM_OBJECTID88', 'AM_OBJECTID89', 'AM_OBJECTID90', 'AM_OBJECTID91', 'AM_OBJECTID93', 'AM_OBJECTID94', 'AM_OBJECTID97', 'AM_OBJECTID98', 'AM_OBJECTID99']
Next objectID: AM_OBJECTID100
Next objectID: AM_OBJECTID62
Next objectID: AM_OBJECTID64
Next objectID: AM_OBJECTID65
Next objectID: AM_OBJECTID66
Next objectID: AM_OBJECTID68
Next objectID: AM_OBJECTID69
Next objectID: AM_OBJECTID72
Next objectID: AM_OBJECTID77
Next objectID: AM_OBJECTID78
Next objectID: AM_OBJECTID79
Next objectID: AM_OBJECTID80
Next objec

In [ ]:
%cat /Users/brodzik/projects/CHARIS/derived_hypsometries/MODSCAG_GF_v09_fromFile_rainfall_less_ET/SY_v01.2001.partition_area.txt

In [ ]:
fig, axes = plt.subplots(1)
for id in majorBasinIDs:
    outFile = myEnv.hypsometry_filename(type='area_by_elevation', drainageID=id)
    print(outFile)
    hyps = Hypsometry(outFile)
    hyps.data.loc['NoDate'].plot(label=id)
    axes.legend()

In [ ]:
fig, axes = plt.subplots(1)
for id in majorBasinIDs:
    outFile = myEnv.hypsometry_filename(type='area_by_elevation', drainageID=id)
    print(outFile)
    hyps = Hypsometry(outFile)
    hyps.data.loc['NoDate'].plot(label=id)
    axes.legend()


In [ ]:
#hyps.data
